# LangChain 1.X.X+ – New Syntax: Create_Agent, Messages, Structured Output, Memory, Middleware, Streaming, MCP

This notebook is an **additional supplement** to the course – it demonstrates several key elements of the new LangChain (1.x) syntax, particularly the API around `create_agent`.
The course uses examples using langchain_classic in many places, which can be replaced by the simplified LangChain syntax in versions above 1.x.

### Instalacja bibliotek

In [1]:
!pip install -q langchain python-dotenv langchain_mcp_adapters fastmcp

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### create_agent

In the new syntax, the agent is created by `create_agent(model=..., tools=[...], ...)` and invoked by `invoke()`.

In [3]:
from langchain.agents import create_agent

def rate_city(city: str) -> str:
    """Rate the city."""
    return f"{city} is the best place in the world!"

agent = create_agent(
    model="gpt-5-mini",
    tools=[rate_city],
    system_prompt="You are a helpful assistant",
)

result = agent.invoke({"messages": [{"role": "user", "content": "Is Poznań a nice city?."}]})

last_msg = result["messages"][-1]
print(last_msg.content)


Short answer: yes — Poznań is widely regarded as a very pleasant city to visit, study in, and live in.

Quick overview
- Historic and compact: a beautifully preserved Old Town (Stary Rynek) with colorful merchant houses, the Town Hall with the famous billy goats (koziołki), and the cathedral on Ostrów Tumski.
- Cultural life: good museums, theaters, regular festivals and events (and the Poznań International Fair brings plenty of business and conference activity).
- Student city: several universities (e.g., Adam Mickiewicz University) give it a lively, youthful atmosphere and decent nightlife.
- Parks and outdoors: Citadel Park, Malta Lake (water sports, walking/running routes), and green neighborhoods make it easy to get outside.
- Transport and accessibility: reliable trams and buses, a regional airport with European connections, and good train links to Warsaw, Berlin, and other cities.
- Food and local flavor: try rogale świętomarcińskie (St. Martin’s croissants) and local cuisine; g

### Messages – Message Objects

LangChain standardizes messages as objects (`SystemMessage`, `HumanMessage`, `AIMessage`, `ToolMessage`) and allows them to be used with chat models.

In [4]:
from langchain.chat_models import init_chat_model
from langchain.messages import SystemMessage, HumanMessage

chat = init_chat_model("gpt-5-mini")

messages = [
    SystemMessage("You are a concise assistant."),
    HumanMessage("Write a 1-sentence summary of what LangChain is."),
]

ai_msg = chat.invoke(messages)  # -> AIMessage
print(ai_msg.content)

LangChain is an open-source framework for building applications with large language models by providing modular components—prompts, chains, agents, memory, and integrations—that connect LLMs to data, tools, and workflows.


### Structured output – `response_format` in `create_agent`

Instead of parsing text, you can ask the agent to return data that conforms to a type (e.g., a Pydantic model).
The result is then sent to `result["structured_response"]`.

In [5]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent

class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address")
    phone: str = Field(description="The phone number")

agent = create_agent(
    model="gpt-5-mini",
    response_format=ContactInfo,
)

result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}
    ]
})

structured = result["structured_response"]
print(structured)
print(type(structured))


name='John Doe' email='john@example.com' phone='(555) 123-4567'
<class '__main__.ContactInfo'>


### Short‑term memory

In LangChain 1.X.X, *short‑term* memory is implemented by a dedicated InMemorySaver component.

- `InMemorySaver()` keeps state in the process's memory (ideal for notebooks)
- `thread_id` identifies the "conversation thread" (important for multiple `invoke()`)


In [6]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_agent(
    model="gpt-5-mini",
    checkpointer=checkpointer,
)

config = {"configurable": {"thread_id": "demo-thread-1"}}

agent.invoke({"messages": [{"role": "user", "content": "Hi! My name is Michael."}]}, config=config)
result = agent.invoke({"messages": [{"role": "user", "content": "What is my name?"}]}, config=config)
print(result["messages"][-1].content)


Your name is Michael.


### Middleware - Human‑in‑the‑loop middleware

Middleware allows you to "hook in" to the agent's execution.
Typical scenario: the agent can *read* data, but actions like *sending email/writing to the database* require approval.
Human‑in‑the‑loop – stopping before running selected tools and resuming with `Command(resume=...)`.

In [7]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command

def read_email(email_id: str) -> str:
    """Read email mock function"""
    return f"(mock) Email content for id={email_id}"

def send_email(recipient: str, subject: str, body: str) -> str:
    """Send email mock function"""
    return f"(mock) Sent email to {recipient} with subject={subject} and content={body}"

checkpointer = InMemorySaver()

agent = create_agent(
    model="gpt-5-mini",
    tools=[read_email, send_email],
    checkpointer=checkpointer,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email": {"allowed_decisions": ["approve", "edit", "reject"]},
                "read_email": False,
            }
        )
    ],
)

config = {"configurable": {"thread_id": "hitl-demo"}}

paused = agent.invoke(
    {"messages": [{"role": "user", "content": "Send an email to alice@example.com with subject 'Hi' and say hello."}]},
    config=config,
)
print("Paused state keys:", paused.keys())

Paused state keys: dict_keys(['messages', '__interrupt__'])


In [8]:
resumed = agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}),
    config=config,
)
print(resumed["messages"][-1].content)

Done — I sent the email to alice@example.com with subject "Hi" and body "Hello."


### Middleware - Guardrails – PII middleware

We can also use guardrails as middleware, for example, to protect against sensitive data leaks in LLM responses.

In [9]:
from langchain.agents import create_agent
from langchain.agents.middleware import PIIMiddleware

def echo(text: str) -> str:
    """Print text."""
    return text

agent = create_agent(
    model="gpt-5-mini",
    tools=[echo],
    middleware=[
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware("credit_card", strategy="mask", apply_to_input=True),
        PIIMiddleware(
            "api_key",
            detector=r"sk-[a-zA-Z0-9]{32}",
            strategy="block",
            apply_to_input=True,
        ),
    ],
)

out = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Extract information from text: My email is john@example.com and card is 5105-1051-0510-5100"
    }]
})
print(out["messages"][-1].content)


{
  "email": "[REDACTED_EMAIL]",
  "card_masked": "****-****-****-5100",
  "card_last4": "5100"
}


### Streaming

In LangChain, we can return partial responses or subsequent generated tokens "on the fly":
- **agent progress** (`stream_mode="updates"`) – event after each step
- **tokens/model messages** (`stream_mode="messages"`) – UI-friendly

In [10]:
from langchain.agents import create_agent

def rate_city(city: str) -> str:
    """Rate city mock tool."""
    return f"The best city is {city}!"

agent = create_agent(model="gpt-5", tools=[rate_city])

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "Is Poznań a nice city?  Rate city and afterwards plan a trip to Poznań in 5 stages."}]},
    stream_mode="updates",
):
    for step, data in chunk.items():
        last = data["messages"][-1]
        print(f"step: {step:>6} | type={type(last).__name__}")
        try:
            print("content_blocks:", last.content_blocks)
        except Exception:
            print("content:", getattr(last, "content", None))


step:  model | type=AIMessage
content_blocks: [{'type': 'tool_call', 'id': 'call_WFpN8AvSTKhA2c38YpgK6zxG', 'name': 'rate_city', 'args': {'city': 'Poznań'}}]
step:  tools | type=ToolMessage
content_blocks: [{'type': 'text', 'text': 'The best city is Poznań!'}]
step:  model | type=AIMessage
content_blocks: [{'type': 'text', 'text': 'Yes—many visitors and residents consider Poznań a very nice city. Quick highlights and caveats:\n\nPros\n- Beautiful Old Town with a Renaissance town hall and lively Stary Rynek.\n- Green spaces like Cytadela Park and Lake Malta, plus Cathedral Island.\n- Vibrant food and café scene (try St. Martin croissants) and good craft beer.\n- Student energy, cultural events (Malta Festival, St. Martin’s Day), and solid museums.\n- Generally affordable compared to Warsaw/Kraków, with efficient trams and good cycling paths.\n\nCons\n- Winters can be gray and chilly; occasional air-quality dips.\n- English is common in the center but less so in outer neighborhoods.\n- F

### MCP – Model Context Protocol

MCP standardizes the use of external tools through LLM.
At LangChain, we use the `langchain-mcp-adapters` adapter and the `MultiServerMCPClient` client, which can retrieve a list of tools from multiple servers.

#### Minimal MCP server (FastMCP)
Save the code below as `math_server.py` in same folder as this notebook.

```python
from fastmcp import FastMCP

mcp = FastMCP("Math")

@mcp.tool()
def add(a: int, b: int) -> int:
    "Add two numbers"
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    "Multiply two numbers"
    return a * b

if __name__ == "__main__":
    mcp.run(transport="stdio")
```

and run cell below:

In [15]:
import asyncio
import nest_asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent

nest_asyncio.apply()

async def demo_mcp():
    client = MultiServerMCPClient(
        {
            "math": {
                "transport": "stdio",
                "command": "python",
                "args": ["math_server.py"],
            },
        }
    )
    tools = await client.get_tools()
    agent = create_agent("gpt-5-mini", tools)

    r1 = await agent.ainvoke({"messages": [{"role": "user", "content": "what's (3 + 5) x 12?"}]})
    print(r1["messages"][-1].content)

# Uncomment below to run
# asyncio.run(demo_mcp())

(3 + 5) = 8, and 8 × 12 = 96.
